Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [2]:
from sqlalchemy import *
import datetime

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [3]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

2025-07-17 10:14:25,314 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-17 10:14:25,315 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("queries")
2025-07-17 10:14:25,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-17 10:14:25,317 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("queries")
2025-07-17 10:14:25,317 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-17 10:14:25,319 INFO sqlalchemy.engine.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2025-07-17 10:14:25,320 INFO sqlalchemy.engine.Engine [no key 0.00066s] ()
2025-07-17 10:14:25,321 INFO sqlalchemy.engine.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [4]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp) VALUES (:id, :keywords, :timestamp)'

In [5]:
import datetime
data = [
    {'keywords':'puppies', 'timestamp': datetime.date(2008, 12, 6) },
    {'keywords':'kittens', 'timestamp': datetime.date(2009, 12, 6) },
    {'keywords':'babies', 'timestamp': datetime.date(2010, 12, 6) },
]

In [6]:
db_conn = engine.connect()
for item in data:
    insert_stmt = queries.insert().values(item)
    result = db_conn.execute(insert_stmt)
    print(result.inserted_primary_key) # print out the primary key it was assigned

2025-07-17 10:14:38,774 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-17 10:14:38,775 INFO sqlalchemy.engine.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2025-07-17 10:14:38,776 INFO sqlalchemy.engine.Engine [generated in 0.00186s] ('puppies', '2008-12-06 00:00:00.000000')
(1,)
2025-07-17 10:14:38,777 INFO sqlalchemy.engine.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2025-07-17 10:14:38,778 INFO sqlalchemy.engine.Engine [cached since 0.004142s ago] ('kittens', '2009-12-06 00:00:00.000000')
(2,)
2025-07-17 10:14:38,779 INFO sqlalchemy.engine.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2025-07-17 10:14:38,780 INFO sqlalchemy.engine.Engine [cached since 0.005619s ago] ('babies', '2010-12-06 00:00:00.000000')
(3,)


In [7]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

2025-07-17 10:14:42,691 INFO sqlalchemy.engine.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2025-07-17 10:14:42,692 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ('kittens', '2025-07-17 10:14:42.691229')


(4,)

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [8]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print(row)

ArgumentError: Column expression, FROM clause, or other columns clause element expected, got [Table('queries', MetaData(), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>, nullable=False), Column('timestamp', DateTime(), table=<queries>, default=CallableColumnDefault(<function datetime.now at 0x1050ae670>)), schema=None)]. Did you mean to say select(Table('queries', MetaData(), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>, nullable=False), Column('timestamp', DateTime(), table=<queries>, default=CallableColumnDefault(<function datetime.now at 0x1050ae670>)), schema=None))?

In [23]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print (row)

ArgumentError: Column expression, FROM clause, or other columns clause element expected, got [Table('queries', MetaData(), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>, nullable=False), Column('timestamp', DateTime(), table=<queries>, default=CallableColumnDefault(<function datetime.now at 0x1050ae670>)), schema=None)]. Did you mean to say select(Table('queries', MetaData(), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>, nullable=False), Column('timestamp', DateTime(), table=<queries>, default=CallableColumnDefault(<function datetime.now at 0x1050ae670>)), schema=None))?

In [ ]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [10]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

/var/folders/r9/9wsb5cdj2pd9lqqydcx6zb280000gr/T/ipykernel_1504/1601245099.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [11]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

Table('queries', MetaData(), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=CallableColumnDefault(<function datetime.now at 0x1066c4ca0>)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [12]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [13]:
query = Query(keywords="iguana")
query.keywords

'iguana'

In [14]:
my_session.add(query)
my_session.commit()
query.id

1

In [15]:
query

<Query(keywords='iguana')>

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [19]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print (q)

<Query(keywords='iguana')>
<Query(keywords='robot')>
<Query(keywords='puppy')>


In [17]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [20]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print (q)

<Query(keywords='iguana')>
<Query(keywords='robot')>
<Query(keywords='puppy')>


In [22]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print (q)

<Query(keywords='robot')>
